# Comparison of SIHR vs SIR

This notebook compares the SIR model against the SIHR model for the initial settings of the `penn_chime` module.
It contains comparisons for

1. no social distancing policies infinite hospital bed capacities
2. no social distancing policies finite hospital bed capacities
3. social distancing policies infinite hospital bed capacities
4. social distancing policies finite hospital bed capacities

**Conclusion:** SIHR predicts significantly more hospitalizations (factor two with a delay of ~ 10 days for `penn_chime` parameters, if hospital capacities are infinite) if one forces models to agree on early admissions per day.

## Details

Effectively, the SIHR model has two parameters which are not present in the SIR model. The hospitalization growth $\beta_H$ and the recovery rate $\gamma_H$. For benchmark purposes, this comparison assumes that the recovery rate in hospital is equal to the rate if not in hospital $\gamma_I$.

Thus, this model has the same number of parameters as the SIR model. With the difference that CHIME computes new hospitalizations using a direct relation to new infections
$$
    \dot H(t) = p_H \dot I(t)
$$
while SIHR uses
$$
    \dot H(t) = \beta_H I(t)
$$
To infer $\beta_H$ for a given $p_H$, I run a fit over the first 3 days.

# Init

## Imports

In [ ]:
from datetime import date

from pandas import DataFrame, read_html
from numpy import zeros, inf

from gvar import gvar
from lsqfit import nonlinear_fit

from penn_chime.model.parameters import Parameters, Disposition
from penn_chime.model.sir import Sir

from models import sir_step, sihr_step, FitFcn

from utils.plotting import plot_sir_sihr_comparison

## Set up Penn CHIME model parameters

In [ ]:
p = Parameters(
    current_hospitalized=69,
    date_first_hospitalized=date(2020, 3, 7),
    doubling_time=4.0,
    hospitalized=Disposition.create(days=7, rate=0.025),
    icu=Disposition.create(days=9, rate=0.0075),
    infectious_days=14,
    market_share=0.15,
    n_days=100,
    population=3600000,
    recovered=0,
    relative_contact_rate=0.3,
    ventilated=Disposition.create(days=10, rate=0.005),
)

p.doubling_time = None
simsir = Sir(p)

## Set up the SIR model for a given set of parameters

Compute values using new fit function

In [ ]:
pars_sir = {
    "beta_i": simsir.beta * p.population,
    "gamma_i": simsir.gamma,
    "initial_susceptible": 3567845,
    "initial_infected": 20737,
    "initial_hospitalized": 0,
    "initial_recovered": 0,
    "hospitalization_rate": 0.05,  # ignore market share
}
x_sir = {
    "n_iter": simsir.raw_df.shape[0],
}


f_sir = FitFcn(sir_step)
df_sir = f_sir(x_sir, pars_sir).iloc[1:]

Check that difference is consistent with zero

In [ ]:
df_sir.head()

## Set up SIHR model for similar parameters to fit $\beta_H$

To fit $\beta_H$, I set all input parameters for the SIHR model besides $\beta_H$ to be equal to the SIR parameters (with a standard derivation of $1\%$ of the mean value).

The `gvar` objects are Gaussian random variables which are initialized with their mean and standard derivation.

In [ ]:
pars_sihr = {
    key: gvar(val, val / 100 if val > 0 else 1.0e-3) for key, val in pars_sir.items()
}
pars_sihr["gamma_h"] = gvar(pars_sihr["gamma_i"])

# Fit this parameter to match SIR
pars_sihr["beta_h"] = gvar(0.1, 0.3)

 Furthermore I only fit new hospitalizations from day 1-3.

In [ ]:
x_sihr = x_sir.copy()
x_sihr["n_iter"] = 4  # four model iterations (including day 0)
x_sihr["capacity"] = inf  # does not play a role for this range

y_sir = df_sir["hospitalized_new"].iloc[:3].values
y_sir = gvar(y_sir, y_sir / 100)  # Assume values are very precise

# Run a sihr step, return only columns `hospitalized_new` as an array and drop the first row
f_sihr = FitFcn(sihr_step, columns=["hospitalized_new"], drop_rows=[0], as_array=True)
fit = nonlinear_fit(data=(x_sihr, y_sir), fcn=f_sihr, prior=pars_sihr)
print(fit)

# Comparisons

## No social distancing, infinite hospital capacity

In [ ]:
f_sihr = FitFcn(sihr_step)

x_sihr = x_sir.copy()
x_sihr["capacity"] = inf

df_sihr = f_sihr(x_sihr, fit.p).iloc[1:]
df_sihr["infected_inclusive"] = df_sihr["infected"] + df_sihr["hospitalized"]

fig = plot_sir_sihr_comparison(df_sir, df_sihr, x_sihr["capacity"])
fig.show()

## No social distancing, finite hospital capacity

In [ ]:
f_sihr = FitFcn(sihr_step)

x_sihr = x_sir.copy()
x_sihr["capacity"] = 7683

df_sihr = f_sihr(x_sihr, fit.p).iloc[1:]
df_sihr["infected_inclusive"] = df_sihr["infected"] + df_sihr["hospitalized"]

fig = plot_sir_sihr_comparison(df_sir, df_sihr, x_sihr["capacity"])
fig.show()

## With social distancing, infinite hospital capacity

In [ ]:
POLICIES = simsir.gen_policy(p)


def beta_i_fcn(x_iter, **kwargs):
    out = zeros(len(x_iter))
    ii = 0
    for beta, n_days in POLICIES:
        for _ in range(n_days):
            out[ii] = beta* p.population
            ii += 1

    return out


print(POLICIES)

In [ ]:
f_sihr_pol = FitFcn(sihr_step, beta_i_fcn=beta_i_fcn)

x_sihr = x_sir.copy()
x_sihr["capacity"] = inf

df_sihr_pol = f_sihr_pol(x_sihr, fit.p).iloc[1:]
df_sihr_pol["infected_inclusive"] = (
    df_sihr_pol["infected"] + df_sihr_pol["hospitalized"]
)

f_sir_pol = FitFcn(sir_step, beta_i_fcn=beta_i_fcn)
df_sir_pol = f_sir_pol(x_sir, pars_sir).iloc[1:]

fig = plot_sir_sihr_comparison(df_sir_pol, df_sihr_pol, x_sihr["capacity"])
fig.show()

## With social distancing, finite hospital capacity

In [ ]:
f_sihr_pol = FitFcn(sihr_step, beta_i_fcn=beta_i_fcn)

x_sihr = x_sir.copy()
x_sihr["capacity"] = 7683

df_sihr_pol = f_sihr_pol(x_sihr, fit.p).iloc[1:]
df_sihr_pol["infected_inclusive"] = (
    df_sihr_pol["infected"] + df_sihr_pol["hospitalized"]
)

f_sir_pol = FitFcn(sir_step, beta_i_fcn=beta_i_fcn)
df_sir_pol = f_sir_pol(x_sir, pars_sir).iloc[1:]

fig = plot_sir_sihr_comparison(df_sir_pol, df_sihr_pol, x_sihr["capacity"])
fig.show()